# Routing

Routing allows you to define routes between component ports


In [ ]:
import pp

In [ ]:
c = pp.Component()
mmi1 = c << pp.c.mmi1x2()
mmi2 = c << pp.c.mmi1x2()
mmi2.move((100, 50))
pp.qp(c)

# Connect strip

Connect strip connects using Manhattan routing

In [ ]:
from pp.routing.connect import connect_strip, connect_strip_way_points

In [ ]:
connect_strip?

In [ ]:
c = pp.Component()
mmi1 = c << pp.c.mmi1x2()
mmi2 = c << pp.c.mmi1x2()
mmi2.move((100, 50))
r = connect_strip(mmi1.ports["E1"], mmi2.ports["W0"])
c.add(r)
pp.qp(c)
pp.show(c)

### Connect strip: Problem

sometimes there are obstacles that connect strip does not see!

In [ ]:
c = pp.Component()
mmi1 = c << pp.c.mmi1x2()
mmi2 = c << pp.c.mmi1x2()
mmi2.move((100, 50))
x = c << pp.c.cross(length=20)
x.move((135, 20))
r = connect_strip(mmi1.ports["E1"], mmi2.ports["E1"])
c.add(r)
pp.qp(c)
pp.show(c)

# Solution: Connect strip way points

You can also specify the points along the route

In [ ]:
connect_strip_way_points?

In [ ]:
connect_strip_way_points([(x0, y0), (x2 + 40, y0), (x2 + 40, y2), (x2, y2)])
mmi2.move((100, 50))
x = c << pp.c.cross(length=20)
x.move((135, 20))

x0 = mmi1.ports["E0"].x
y0 = mmi1.ports["E0"].y


x2 = mmi2.ports["E0"].x
y2 = mmi2.ports["E0"].y

r2 = connect_strip_way_points([(x0, y0), (x2 + 40, y0), (x2 + 40, y2), (x2, y2)])
c.add(r2)
pp.qp(c)
pp.show(c)

# River routing

**Problem**: when connecting lots of ports there are collisions

In [ ]:
xs_top = [0, 10, 20, 40, 50, 80]
pitch = 127.0
N = len(xs_top)
xs_bottom = [(i - N / 2) * pitch for i in range(N)]

top_ports = [pp.Port("top_{}".format(i), (xs_top[i], 0), 0.5, 270) for i in range(N)]

bottom_ports = [
    pp.Port("bottom_{}".format(i), (xs_bottom[i], -400), 0.5, 90) for i in range(N)
]

top_cell = pp.Component(name="connect_bundle")

for p1, p2 in zip(top_ports, bottom_ports):
    r = pp.routing.connect_strip(p1, p2)
    top_cell.add(r)
top_cell.name = "connect_bundle"
pp.qp(top_cell)
pp.show(top_cell)

**solution**: river routing routes several ports without collisions

In [ ]:
xs_top = [0, 10, 20, 40, 50, 80]
pitch = 127.0
N = len(xs_top)
xs_bottom = [(i - N / 2) * pitch for i in range(N)]

top_ports = [pp.Port("top_{}".format(i), (xs_top[i], 0), 0.5, 270) for i in range(N)]

bottom_ports = [
    pp.Port("bottom_{}".format(i), (xs_bottom[i], -400), 0.5, 90) for i in range(N)
]

top_cell = pp.Component(name="connect_bundle")
elements = pp.routing.connect_bundle(top_ports, bottom_ports)
for e in elements:
    top_cell.add(e)
top_cell.name = "connect_bundle"
pp.qp(top_cell)
pp.show(top_cell)

# Connect electrical and optical ports

In [ ]:
import pp

mzi = pp.c.mzi2x2(with_elec_connections=True)
pp.qp(mzi)

In [ ]:
pp.c.pad_array?

In [ ]:
pads = pp.c.pad_array(n=3, port_list=["S"])

In [ ]:
pp.routing.link_electrical_ports?

In [ ]:
c = pp.Component("mzi_with_pads")
p = c << pads
c << mzi

p.move((-150, 300))
pp.qp(c)

In [ ]:
routes = pp.routing.link_electrical_ports(
    list(p.ports.values()), mzi.get_electrical_ports()
)

In [ ]:
c.add(routes)
pp.qp(c)
pp.show(c)

In [ ]:
c = pp.Component("mzi_with_pads")
p = c << pads
c << mzi

p.move((-150, 300))
pp.qp(c)

In [ ]:
from pp.routing.connect import connect_elec_waypoints


routes = pp.routing.connect_bundle(
    list(p.ports.values()),
    mzi.get_electrical_ports(),
    route_filter=connect_elec_waypoints,
    bend_radius=10,
)

In [ ]:
pp.qp(routes)

In [ ]:
import pp

c = pp.c.mzi2x2(with_elec_connections=True)
cc = pp.routing.add_electrical_pads_top(c)
pp.show(cc)
pp.qp(cc)

In [ ]:
import pp

c = pp.c.mzi2x2(with_elec_connections=True)
cc = pp.routing.add_electrical_pads_shortest(c)
pp.show(cc)
pp.qp(cc)

In [ ]:
import pp

c = pp.c.mzi2x2(with_elec_connections=True)
cc = pp.routing.add_electrical_pads(c, fanout_length=100)
pp.show(cc)
pp.qp(cc)

In [ ]:
import pp

c = pp.c.cross(length=100, layer=pp.LAYER.M3, port_type="dc")
c.move((20, 50))
cc = pp.routing.add_electrical_pads_shortest(c)
pp.show(cc)
pp.qp(cc)